In [2]:
import gradio as gr
import sqlite3
import json

# Database setup
def init_db():
    conn = sqlite3.connect("chat_history.db")
    cursor = conn.cursor()
    cursor.execute("INSERT OR IGNORE INTO models (model_id, model_name) VALUES (1, 'Llama 3.2')")
    cursor.execute("INSERT OR IGNORE INTO models (model_id, model_name) VALUES (2, 'Phi 3.5')")
    conn = sqlite3.connect("chat_history.db")
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS chats (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        session_name TEXT,
                        history TEXT
                    )''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS models (
                        model_id INTEGER PRIMARY KEY AUTOINCREMENT,
                        model_name TEXT
                    )''')
    conn.commit()
    conn.close()

init_db()

# Fetch past conversations
def get_past_conversations():
    conn = sqlite3.connect("chat_history.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id, session_name FROM chats")
    sessions = cursor.fetchall()
    conn.close()
    return {str(s[0]): f"Conversation {s[0]}" for s in sessions}

# Fetching available models
def get_model_names():
    conn = sqlite3.connect("chat_history.db")
    cursor = conn.cursor()
    cursor.execute("SELECT model_id, model_name FROM models")
    models = cursor.fetchall()
    conn.close()
    return {str(m[0]): m[1] for m in models}

# Loading chat history
def load_chat(session_id):
    conn = sqlite3.connect("chat_history.db")
    cursor = conn.cursor()
    cursor.execute("SELECT history FROM chats WHERE id=?", (session_id,))
    history = cursor.fetchone()
    conn.close()
    
    if history:
        try:
            loaded_history = json.loads(history[0])
            if isinstance(loaded_history, list) and all(isinstance(item, list) and len(item) == 2 for item in loaded_history):
                return loaded_history
        except json.JSONDecodeError:
            return []
    
    return []

# Saved conversation
def save_chat(session_id, messages):
    conn = sqlite3.connect("chat_history.db")
    cursor = conn.cursor()
    cursor.execute("UPDATE chats SET history = ? WHERE id = ?", (json.dumps(messages), session_id))
    conn.commit()
    conn.close()

# Chatbot response
def chatbot_response(user_input, history, session_id):
    if history is None:
        history = []

    if not isinstance(history, list):
        history = []

    if isinstance(history, list) and all(isinstance(item, list) and len(item) == 2 for item in history):
        pass
    else:
        history = []

    bot_response = "Here will be the response from trained model"
    new_entry = [user_input, bot_response]
    history.append(new_entry)
    
    save_chat(session_id, history)

    return history, history, ""

# UI Components
def chat_interface():
    past_conversations = get_past_conversations()
    available_models = get_model_names()
    with gr.Blocks() as demo:
        gr.HTML("<h1 style='text-align: center;'>COT-Reasoning GPT</h1>")
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("## Past Conversations")
                session_list = gr.Dropdown(choices=[(f"Conversation {key}", key) for key in past_conversations.keys()], label="Select Conversation")
                model_list = gr.Dropdown(choices=[(name, key) for key, name in available_models.items()], label="Select Model")
                load_button = gr.Button("Load Chat")
                new_chat_button = gr.Button("Start New Chat")
            
            with gr.Column(scale=3):
                chatbot = gr.Chatbot()
                user_input = gr.Textbox(label="Type your message")
                send_button = gr.Button("Send")
                
        def load_selected_chat(session_label):
            session_id = session_label.split(" ")[-1]
            history = load_chat(session_id)
            return history if history else [], session_id
        
        def start_new_chat():
            conn = sqlite3.connect("chat_history.db")
            cursor = conn.cursor()
            cursor.execute("INSERT INTO chats (session_name, history) VALUES (?, ?)", ("Conversation", json.dumps([])))
            conn.commit()
            new_session_id = cursor.lastrowid
            conn.close()
            updated_conversations = get_past_conversations()
            return gr.Dropdown(choices=[(f"Conversation {key}", key) for key in updated_conversations.keys()]), []
        
        load_button.click(load_selected_chat, inputs=[session_list], outputs=[chatbot, session_list])
        send_button.click(chatbot_response, inputs=[user_input, chatbot, session_list], outputs=[chatbot, chatbot, user_input])
        new_chat_button.click(start_new_chat, outputs=[session_list, chatbot])
    
    return demo

if __name__ == "__main__":
    chat_interface().launch(share=True)


/Users/saumyavarshney/Documents/MSDA/SEMESTER - SPRING'25/DATA 298B - MSDA Project/Front-End Gradio/.venv/lib/python3.11/site-packages/gradio/components/chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://4246c7268dda81de27.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
